In [1]:
import json
import geopandas as gpd
from pathlib import Path

# ---- Paths ----
GEOJSON_PATH = "../highway_network/uk_driving_edges_simplified.geojson"
GT_PATH      = "GT_AADT_8years.json"   # or your latest GT file
OUT_PATH     = "edge_to_highway_type.json"

# ---- Load GT edges (keys only) ----
with open(GT_PATH, "r") as f:
    gt = json.load(f)

gt_edge_ids = set(gt.keys())

# ---- Load simplified network ----
gdf = gpd.read_file(GEOJSON_PATH)

# Ensure columns are correct
gdf["u"] = gdf["u"].astype(str)
gdf["v"] = gdf["v"].astype(str)
gdf["key"] = gdf["key"].astype(str)

# Construct edge_id
gdf["edge_id"] = gdf["u"] + "_" + gdf["v"] + "_" + gdf["key"]

# Normalize highway tags
gdf["highway"] = gdf["highway"].astype(str).str.lower()

# ---- Filter to only GT edges ----
gdf_gt = gdf[gdf["edge_id"].isin(gt_edge_ids)]

# ---- Build mapping edge_id -> highway tag ----
edge_to_highway = dict(zip(gdf_gt["edge_id"], gdf_gt["highway"]))

# ---- Save JSON ----
Path(OUT_PATH).parent.mkdir(parents=True, exist_ok=True)
with open(OUT_PATH, "w") as f:
    json.dump(edge_to_highway, f, indent=2)

print(f"Saved: {OUT_PATH}  ({len(edge_to_highway)} edges)")

Saved: edge_to_highway_type.json  (5088 edges)


In [2]:
import json
from collections import Counter

IN_PATH = "edge_to_highway_type.json"

with open(IN_PATH, "r") as f:
    edge_to_highway = json.load(f)

cnt = Counter(edge_to_highway.values())

print("Highway type counts:")
for h, n in cnt.most_common():
    print(f"{h:20s} {n}")

Highway type counts:
motorway             1844
trunk                1544
motorway_link        1244
trunk_link           456
